In [1]:
# pip install pygsheets
# pip install pymssql

import pandas as pd
import pygsheets
import calendar
import numpy as np
import sys
# from pyhive import presto
# import psycopg2
from datetime import datetime,timedelta,date
import logging
import time
import glob
import datetime as dt

logging.basicConfig(level=logging.INFO, format="%(asctime)-15s %(levelname)-8s %(message)s")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

service_file = './test-367816-470c46e2b23a.json'gc = pygsheets.authorize(service_file=service_file)

import pymssql

def exp_backoff(conn, query, queryName, updateQuery=False, commit = True):
        exp_time = [32, 64, 128, 256, 512, 0]
        resultDataframe = pd.DataFrame()
        
        for sleepTime in exp_time:
            try:
                logging.info("Executing query...")
                start_time = time.time()
                cursor = conn.cursor()
                logging.debug("Query: %s"%(query))
                cursor.execute(query)
                if updateQuery==False:
                    resultDataframe = pd.DataFrame(cursor.fetchall(), columns = [desc[0] for desc in cursor.description])
                    dim = resultDataframe.shape
                else:
                    dim = cursor.rowcount
                if updateQuery and commit:
                    conn.commit()
                elapsed_time = time.time() - start_time
                logging.info("Query: %s | Time Taken: %s mins and %s secs | Dimension: %s"%(queryName,int(elapsed_time/60),int(elapsed_time%60),dim))
                if updateQuery==False:
                    return resultDataframe
                return
            except:
                logging.error("Unexpected error: %s"%(sys.exc_info()[0]))
                logging.error(traceback.print_exc())
                logging.error("Retrying %s in %s seconds"%(queryName, sleepTime))
                time.sleep(sleepTime)
        
        logging.error("Exceded no. of tries....")
        sys.exit()
        

conn = pymssql.connect(server='PB-PROD-SEC1',user='Anantjindal',password='Anant@406&#!Live#@')
# sh=gc.open_by_key("1H4PF2EuoFGntZLr3-GEi_CY94k4zRHiSdTG6ooTSlIc")
# city_cat=sh.worksheet_by_title("All")
# city_cat = pd.DataFrame(city_cat.get_as_df(numerize=True))
# city_cat = city_cat[["city","city_cat"]]
# city_cat.columns

In [2]:
supplier_query = """select rl.SupplierID, count (distinct bd.matrixleadid) as 'Case Count'
from healthdb.ren.RenewalLeads rl
inner join Healthdb.ren.ProposerBookingDetails bd on bd.ProposerID=rl.ProposerID
where rl.CreatedOn > GETDATE()-60
and bd.BookingStatus not in (3,6,9)
and bd.matrixleadid not in 
(select distinct matrixleadid from Healthdb.ren.EmailTriggersHistory where createdon > '2022-09-01')
group by  rl.SupplierID
order by 'Case Count' desc"""

supplier_data = exp_backoff(conn,supplier_query,'supplier_query')

# sh = gc.open_by_key('1X3jt5xs_8tXL3H32p4oA0EiHe0mbiDbYdLMkHYMJydQ')
# wks=sh.worksheet_by_title("case_wo_email")
# wks.clear(start='A',end='B')
# wks.set_dataframe(supplier_data,(1,1))

2022-11-20 22:21:42,047 INFO     Executing query...
2022-11-20 22:21:52,579 INFO     Query: supplier_query | Time Taken: 0 mins and 10 secs | Dimension: (7, 2)


In [3]:
trigger_query = """select TriggerName, count (distinct bd.MatrixLeadId) as 'Lead Count'
from healthdb.ren.RenewalLeads rl (nolock)
inner join Healthdb.ren.ProposerBookingDetails bd (nolock) on bd.ProposerID=rl.ProposerID
inner join Healthdb.ren.EmailTriggersHistory eh (nolock) on eh.MatrixLeadID=bd.MatrixLeadId
and applicable=1
where rl.CreatedOn> GETDATE()-90
and convert(date,eh.CreatedOn) = convert(date,GETDATE()-1) and rl.productid=2
group by TriggerName 
order by 'Lead Count' desc"""

trigger_data = exp_backoff(conn,trigger_query,'trigger_query')

sh = gc.open_by_key('1X3jt5xs_8tXL3H32p4oA0EiHe0mbiDbYdLMkHYMJydQ')
wks=sh.worksheet_by_title("trigger")
wks.clear(start='A',end='B')
wks.set_dataframe(trigger_data,(1,1))

2022-11-18 10:27:12,022 INFO     Executing query...
2022-11-18 10:27:13,060 INFO     Query: trigger_query | Time Taken: 0 mins and 1 secs | Dimension: (14, 2)


In [4]:
conn.close()

In [5]:
# import smtplib
# email_conn = smtplib.SMTP('smtp.gmail.com',587)
# print(type(email_conn))
# email_conn.ehlo()
# email_conn.starttls()

# email_conn.login('mayankkanojia@policybazaar.com','yclnckhdmcxdsxvm')

# sender_email = "mayankkanojia@policybazaar.com"
# receiver_email = "harshsingh@policybazaar.com"
# message = """\
# Subject: Hi Harsh

# This message is a test message sent from Python 
# {}.""".format(trigger_data)


# email_conn.sendmail(sender_email,receiver_email,message)

In [6]:
date = str(dt.date.today())
date = datetime.strptime(date, '%Y-%m-%d')
formatted_date = date.strftime('%d-%b-%Y')

In [7]:
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table

def send_mail(body):

    message = MIMEMultipart()
    message['Subject'] = 'Supplier Wise Emails Remaining - {}'.format(formatted_date)
    message['From'] = 'mayankkanojia@policybazaar.com'
#     message['To'] = 'tushargupta@policybazaar.com'
#     message['CC'] = 'rohan@policybazaar.com, jitin@policybazaar.com'
    message['To'] = 'mayankkanojia@policybazaar.com'
    message['CC'] = 'mayankkanojia@policybazaar.com, mayankkanojia@policybazaar.com'
    body_content = body
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()
    server = SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(message['From'], 'yclnckhdmcxdsxvm')
    server.sendmail(message['From'], message['To'], msg_body)
    server.quit()

In [8]:
supplier_data_table = build_table(supplier_data, 'blue_light')
trigger_data_table = build_table(trigger_data, 'blue_light')

In [9]:
body = """Hi All<p><p>
Please find the daily reports on:<p><p>
<b><u>1. Remaining Cases - Supplier Wise</u></b><p><p> 
{}<p><p>
<b><u>2. Lead Count against TriggerName</u></b><p><p> 
{}<p><p>
Regards<br> 
Mayank Kanojia<br>
Lead BA<br>
Product""".format(supplier_data_table,trigger_data_table)
# body1 = MIMEText(body, "html")
# body2 = body1.as_string()

In [10]:
send_mail(body)